# হোটেল বুকিং প্রায়োরিটি মেম্বার মিডলওয়্যার সহ

এই নোটবুকটি **ফাংশন-ভিত্তিক মিডলওয়্যার** ব্যবহার করে Microsoft Agent Framework এর একটি উদাহরণ প্রদর্শন করে। আমরা শর্তাধীন ওয়ার্কফ্লো উদাহরণটির উপর ভিত্তি করে একটি মিডলওয়্যার স্তর যোগ করি যা প্রায়োরিটি মেম্বারদের বিশেষ সুবিধা প্রদান করে।

## আপনি যা শিখবেন:
1. **ফাংশন-ভিত্তিক মিডলওয়্যার**: ফাংশনের ফলাফল আটকানো এবং পরিবর্তন করা
2. **কনটেক্সট অ্যাক্সেস**: `context.result` পড়া এবং সম্পাদনা করা
3. **ব্যবসায়িক লজিক বাস্তবায়ন**: প্রায়োরিটি মেম্বার সুবিধা
4. **ফলাফল ওভাররাইড**: ব্যবহারকারীর স্ট্যাটাস অনুযায়ী ফাংশনের ফলাফল পরিবর্তন করা
5. **একই ওয়ার্কফ্লো, ভিন্ন ফলাফল**: মিডলওয়্যার দ্বারা চালিত আচরণ পরিবর্তন

## মিডলওয়্যার সহ ওয়ার্কফ্লো আর্কিটেকচার:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## শর্তাধীন ওয়ার্কফ্লো থেকে মূল পার্থক্য:

**মিডলওয়্যার ছাড়া** (14-conditional-workflow.ipynb):
- প্যারিসে কোনো রুম নেই → alternative_agent-এ রুট করা হয়

**মিডলওয়্যার সহ** (এই নোটবুক):
- সাধারণ ব্যবহারকারী + প্যারিস → কোনো রুম নেই → alternative_agent-এ রুট করা হয়
- প্রায়োরিটি ব্যবহারকারী + প্যারিস → 🌟 মিডলওয়্যার ওভাররাইড! → রুম উপলব্ধ → booking_agent-এ রুট করা হয়

## পূর্বশর্ত:
- Microsoft Agent Framework ইনস্টল করা
- শর্তাধীন ওয়ার্কফ্লো সম্পর্কে ধারণা (দেখুন 14-conditional-workflow.ipynb)
- GitHub টোকেন বা OpenAI API কী
- মিডলওয়্যার প্যাটার্ন সম্পর্কে মৌলিক ধারণা


In [2]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## ধাপ ১: কাঠামোগত আউটপুটের জন্য Pydantic মডেল সংজ্ঞায়িত করুন

এই মডেলগুলো এজেন্টদের ফেরত দেওয়া **স্কিমা** সংজ্ঞায়িত করে। আমরা একটি `priority_override` ক্ষেত্র যোগ করেছি, যা মধ্যস্থতাকারী যখন প্রাপ্যতার ফলাফল পরিবর্তন করে তখন তা ট্র্যাক করে।


In [3]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## ধাপ ২: প্রায়োরিটি মেম্বারদের ডাটাবেস সংজ্ঞায়িত করুন

এই ডেমোর জন্য, আমরা একটি প্রায়োরিটি মেম্বারশিপ ডাটাবেস সিমুলেট করব। প্রোডাকশনে, এটি একটি বাস্তব ডাটাবেস বা API থেকে তথ্য সংগ্রহ করবে।

**প্রায়োরিটি মেম্বাররা:**
- `alice@example.com` - VIP সদস্য
- `bob@example.com` - প্রিমিয়াম সদস্য  
- `priority_user` - টেস্ট অ্যাকাউন্ট


In [4]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## ধাপ ৩: হোটেল বুকিং টুল তৈরি করুন

শর্তাধীন ওয়ার্কফ্লোর মতোই, তবে এখন এটি মধ্যস্থতাকারী দ্বারা আটকানো হবে!


In [5]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## ধাপ ৪: 🌟 প্রায়োরিটি চেক মিডলওয়্যার তৈরি করুন (মূল বৈশিষ্ট্য!)

এটি এই নোটবুকের **মূল কার্যকারিতা**। এই মিডলওয়্যারটি:

1. **আটকায়** hotel_booking ফাংশন কল
2. **স্বাভাবিকভাবে কার্যকর করে** ফাংশনটি `next(context)` কল করে
3. **পরীক্ষা করে** `context.result`-এ ফলাফল
4. **ফলাফল ওভাররাইড করে** যদি ব্যবহারকারী প্রায়োরিটি হয় এবং কোনো রুম উপলব্ধ না থাকে
5. **পরিবর্তিত ফলাফলটি** এজেন্টে ফেরত পাঠায়

**মূল প্যাটার্ন:**
```python
async def my_middleware(context, next):
    await next(context)  # Execute function
    # Now context.result contains the function's output
    if some_condition:
        context.result = new_value  # Override!
```


In [6]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## ধাপ ৫: রাউটিংয়ের জন্য শর্ত ফাংশন সংজ্ঞায়িত করুন

শর্ত ফাংশনগুলো শর্তাধীন ওয়ার্কফ্লোর মতোই - এগুলো কাঠামোবদ্ধ আউটপুট পরিদর্শন করে রাউটিং নির্ধারণ করে।


In [7]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## ধাপ ৬: কাস্টম ডিসপ্লে এক্সিকিউটর তৈরি করুন

আগের মতোই এক্সিকিউটর - এটি চূড়ান্ত ওয়ার্কফ্লো আউটপুট প্রদর্শন করে।


In [8]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## ধাপ ৭: পরিবেশ ভেরিয়েবল লোড করুন

LLM ক্লায়েন্ট (GitHub Models বা OpenAI) কনফিগার করুন।


In [10]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## ধাপ ৮: মধ্যস্থতাকারী ব্যবহার করে AI এজেন্ট তৈরি করুন

**মূল পার্থক্য:** যখন availability_agent তৈরি করা হয়, তখন আমরা `middleware` প্যারামিটারটি পাস করি!

এভাবেই আমরা priority_check_middleware-কে এজেন্টের ফাংশন আহ্বানের পাইপলাইনে সংযুক্ত করি।


In [ ]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## ধাপ ৯: ওয়ার্কফ্লো তৈরি করুন

আগের মতো একই ওয়ার্কফ্লো কাঠামো - প্রাপ্যতার উপর ভিত্তি করে শর্তসাপেক্ষ রাউটিং।


In [12]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## ধাপ ১০: টেস্ট কেস ১ - প্যারিসে সাধারণ ব্যবহারকারী (কোনও ওভাররাইড নেই)

একজন সাধারণ ব্যবহারকারী প্যারিসে বুক করার চেষ্টা করেন → কোনও রুম নেই → বিকল্প এজেন্টে রুট করা হয়


In [13]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## ধাপ ১১: টেস্ট কেস ২ - 🌟 প্যারিসে প্রাধান্যপ্রাপ্ত ব্যবহারকারী (ওভাররাইড সহ!)

একজন প্রাধান্যপ্রাপ্ত সদস্য প্যারিস বুক করতে চেষ্টা করেন → প্রথমে কোনো রুম নেই → 🌟 মিডলওয়্যার ওভাররাইড করে! → booking_agent-এ রুট করে

**এটি মিডলওয়্যারের ক্ষমতার মূল প্রদর্শনী!**


In [14]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## ধাপ ১২: টেস্ট কেস ৩ - স্টকহোমে প্রাধান্যপ্রাপ্ত ব্যবহারকারী (ইতিমধ্যেই উপলব্ধ)

প্রাধান্যপ্রাপ্ত ব্যবহারকারী স্টকহোম চেষ্টা করেন → রুম উপলব্ধ → কোনো ওভাররাইড প্রয়োজন নেই → booking_agent-এ রুট হয়

এটি প্রমাণ করে যে middleware শুধুমাত্র প্রয়োজন হলে কাজ করে!


In [15]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## মূল বিষয় এবং মিডলওয়্যার ধারণা

### ✅ আপনি যা শিখেছেন:

#### **1. ফাংশন-ভিত্তিক মিডলওয়্যার প্যাটার্ন**

মিডলওয়্যার একটি সহজ অ্যাসিঙ্ক ফাংশন ব্যবহার করে ফাংশন কলগুলোকে আটকায়:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # Before function execution
    print("Intercepting...")
    
    # Execute the function
    await next(context)
    
    # After function execution - inspect result
    if context.result:
        # Modify result if needed
        context.result = modified_value
```

#### **2. কন্টেক্সট অ্যাক্সেস এবং রেজাল্ট ওভাররাইড**

- `context.function` - কল করা ফাংশনটি অ্যাক্সেস করুন
- `context.arguments` - ফাংশনের আর্গুমেন্টগুলো পড়ুন
- `context.kwargs` - অতিরিক্ত প্যারামিটারগুলো অ্যাক্সেস করুন
- `await next(context)` - ফাংশনটি সম্পাদন করুন
- `context.result` - ফাংশনের আউটপুট পড়ুন/পরিবর্তন করুন

#### **3. বিজনেস লজিক বাস্তবায়ন**

আমাদের মিডলওয়্যার প্রাইরিটি মেম্বার সুবিধা বাস্তবায়ন করে:
- **সাধারণ ব্যবহারকারী**: কোনো পরিবর্তন নেই, সাধারণ ওয়ার্কফ্লো
- **প্রাইরিটি ব্যবহারকারী**: "কোনো অ্যাভেইলেবিলিটি নেই" → "অ্যাভেইলেবল" ওভাররাইড
- **শর্তসাপেক্ষ লজিক**: শুধুমাত্র প্রয়োজন হলে ওভাররাইড করে

#### **4. একই ওয়ার্কফ্লো, ভিন্ন ফলাফল**

মিডলওয়্যারের শক্তি:
- ✅ ওয়ার্কফ্লো স্ট্রাকচারে কোনো পরিবর্তন নেই
- ✅ টুল ফাংশনে কোনো পরিবর্তন নেই
- ✅ শর্তসাপেক্ষ রাউটিং লজিকে কোনো পরিবর্তন নেই
- ✅ শুধু মিডলওয়্যার → ভিন্ন আচরণ!

### 🚀 বাস্তব জীবনের প্রয়োগ:

1. **VIP/প্রিমিয়াম ফিচার**
   - প্রিমিয়াম ব্যবহারকারীদের জন্য রেট লিমিট ওভাররাইড
   - রিসোর্সে প্রাইরিটি অ্যাক্সেস প্রদান
   - প্রিমিয়াম ফিচারগুলো ডায়নামিকভাবে আনলক করুন

2. **A/B টেস্টিং**
   - ব্যবহারকারীদের বিভিন্ন ইমপ্লিমেন্টেশনে রাউট করুন
   - নির্দিষ্ট ব্যবহারকারীদের সাথে নতুন ফিচার পরীক্ষা করুন
   - ধীরে ধীরে ফিচার রোলআউট করুন

3. **সিকিউরিটি ও কমপ্লায়েন্স**
   - ফাংশন কলগুলো অডিট করুন
   - সংবেদনশীল অপারেশনগুলো ব্লক করুন
   - বিজনেস রুলগুলো প্রয়োগ করুন

4. **পারফরম্যান্স অপটিমাইজেশন**
   - নির্দিষ্ট ব্যবহারকারীদের জন্য রেজাল্ট ক্যাশ করুন
   - সম্ভব হলে ব্যয়বহুল অপারেশনগুলো এড়িয়ে চলুন
   - ডায়নামিক রিসোর্স বরাদ্দ করুন

5. **এরর হ্যান্ডলিং ও রিট্রাই**
   - ত্রুটি সুন্দরভাবে ধরুন এবং হ্যান্ডল করুন
   - রিট্রাই লজিক বাস্তবায়ন করুন
   - বিকল্প ইমপ্লিমেন্টেশনে ফ্যালব্যাক করুন

6. **লগিং ও মনিটরিং**
   - ফাংশন এক্সিকিউশন টাইম ট্র্যাক করুন
   - প্যারামিটার এবং রেজাল্ট লগ করুন
   - ব্যবহার প্যাটার্ন মনিটর করুন

### 🔑 ডেকোরেটর থেকে মিডলওয়্যারের মূল পার্থক্য:

| ফিচার | ডেকোরেটর | মিডলওয়্যার |
|-------|-----------|------------|
| **স্কোপ** | একক ফাংশন | এজেন্টের সব ফাংশন |
| **ফ্লেক্সিবিলিটি** | ডেফিনিশনে নির্ধারিত | রানটাইমে ডায়নামিক |
| **কন্টেক্সট** | সীমিত | পূর্ণ এজেন্ট কন্টেক্সট |
| **কম্পোজিশন** | একাধিক ডেকোরেটর | মিডলওয়্যার পাইপলাইন |
| **এজেন্ট-অওয়ার** | না | হ্যাঁ (এজেন্ট স্টেট অ্যাক্সেস) |

### 📚 কখন মিডলওয়্যার ব্যবহার করবেন:

✅ **মিডলওয়্যার ব্যবহার করুন যখন:**
- ব্যবহারকারী/সেশন স্টেটের উপর ভিত্তি করে আচরণ পরিবর্তন করতে চান
- একাধিক ফাংশনে লজিক প্রয়োগ করতে চান
- এজেন্ট-লেভেল কন্টেক্সট অ্যাক্সেস প্রয়োজন
- ক্রস-কাটিং কনসার্ন (লগিং, অথ, ইত্যাদি) বাস্তবায়ন করছেন

❌ **মিডলওয়্যার ব্যবহার করবেন না যখন:**
- সাধারণ ইনপুট ভ্যালিডেশন (Pydantic ব্যবহার করুন)
- ফাংশন-নির্দিষ্ট লজিক (ফাংশনে রাখুন)
- একবারের পরিবর্তন (ফাংশন পরিবর্তন করুন)

### 🎓 উন্নত প্যাটার্ন:

```python
# Multiple middleware (execution order matters!)
middleware=[
    logging_middleware,      # Logs first
    auth_middleware,         # Then checks auth
    cache_middleware,        # Then checks cache
    rate_limit_middleware,   # Then rate limits
    priority_check_middleware  # Finally priority check
]

# Conditional middleware execution
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # Modify result
    else:
        # Skip execution entirely
        context.result = cached_value
```

### 🔗 সম্পর্কিত ধারণা:

- **এজেন্ট মিডলওয়্যার**: এজেন্ট.run() কলগুলো আটকায়
- **ফাংশন মিডলওয়্যার**: টুল ফাংশন কলগুলো আটকায় (আমরা যা ব্যবহার করেছি!)
- **মিডলওয়্যার পাইপলাইন**: ক্রমানুসারে মিডলওয়্যার চেইন সম্পাদন
- **কন্টেক্সট প্রোপাগেশন**: মিডলওয়্যার চেইনের মাধ্যমে স্টেট পাস করুন



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসাধ্য সঠিকতার জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
